# 第2章 点群処理の基礎

## 2.1 ファイルの入出力

- sys.argv[0]：　 スクリプト名
- sys.argv[1:] ：　コマンドライン引数

In [19]:
import sys
import struct

#with open(sys.argv[1], 'rb') as f:
filename = "../3rdparty/Open3D/examples/test_data/bathtub_0154.ply"
with open(filename, 'rb') as f:
    #ヘッダーの読み込み
    while True:
        line = f.readline()
        print(line)
        if b'end_header' in line:
            break
        if b'vertex' in line:
            vnum = int(line.split(b' ')[-1]) #頂点の数
        if b'face' in line:
            fnum = int(line.split(b' ')[-1])#面の数
    
    #頂点の読み込み
    for i in range(vnum):
        for j in range(3):
            print(struct.unpack('f', f.read(4))[0], end=' ')
        print("")
    
    #面の読み込み
    for i in range(fnum):
        n = struct.unpack('B', f,read(1))[0]
        for j in range(n):
            print(struct.unpack('i', f.read(4))[0], end=' ')
        print("")

b'ply\n'
b'format binary_little_endian 1.0\n'
b'comment VCGLIB generated\n'
b'element vertex 1494\n'
b'property float x\n'
b'property float y\n'
b'property float z\n'
b'element face 1194\n'
b'property list uchar int vertex_indices\n'


ValueError: invalid literal for int() with base 10: b'vertex_indices\n'

## 2.2 描画

In [23]:
import sys 
import numpy as np
import open3d as o3d

#filename = sys.argv[1]
#filename = "../3rdparty/Open3D/examples/test_data/bathtub_0154.ply"
#filename = "../3rdparty/Open3D/examples/test_data/fragment.ply"
#filename = "../../Downloads/20221013_work_8.PLY" 
filename = "../../Downloads/Housei_20221027_02.ply"

print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename) #ファイルから点群データを読み込み
#pcd = o3d.io.read_triangle_mesh(filename) #ファイルから点群データをメッシュで読み込み

print(pcd)
print(np.asarray(pcd.points))

o3d.visualization.draw_geometries([pcd]) #描画

Loading a point cloud from ../../Downloads/Housei_20221027_02.ply
PointCloud with 1892989 points.
[[-18.7566452  -32.68566895  63.2834053 ]
 [-16.99583817 -30.5393219   68.82434082]
 [-11.1352911   20.93461609  13.35097408]
 ...
 [-16.6009903   30.10222054   7.86988544]
 [-18.54046631  35.04523468   7.35676718]
 [ -5.76789188 -12.85958672  69.9982605 ]]


ここでOpen3dは点群を描画する関数だから面の情報は無視される  
だから見た目がMeshlabとは異なる

## 2.3 回転・並進・スケール変換  
数式に関しては、後で追う  
  
- Open3Dでは、回転行列Rを用いて回転を指定する

In [3]:
#メッシュにするコード
import sys
import open3d as o3d
import numpy as np

"""
メッシュを回転したり移動したりすることはできるが、最初は灰色で均一に塗られており、「3d」のように見えない。その理由は、この時点のメッシュに頂点や面の法線がないためである。 
したがって、より洗練されたPhongシェーディングの代わりに、均一なカラーシェーディングが使用される。
"""
def ply_to_mesh(filename):
    print("Loading a point cloud from", filename)
    mesh = o3d.io.read_triangle_mesh(filename)
    print(mesh)
    #メッシュでデータを表示
    o3d.visualization.draw_geometries([mesh])

    pcd = o3d.geometry.PointCloud()
    pcd.points = mesh.vertices
    #法線推定して、法線データを出力する
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=10.0, max_nn=10))

    print(np.asarray(pcd.normals))
    o3d.visualization.draw_geometries([pcd], point_show_normal=True)

    #メッシュデータで法線推定して、法線データを出力する
    mesh.compute_vertex_normals()
    mesh.paint_uniform_color([0.5, 0.5, 0.5])
    #print(np.asarray(mesh.triangle_normals))
    #o3d.visualization.draw_geometries([mesh])
    
    return pcd, mesh

In [9]:
import numpy as np
import open3d as o3d
import copy 

#軸のメッシュを作成
#mesh = o3d.geometry.TriangleMesh.create_coordinate_frame()
#filename = "../../Downloads/Housei_20221027_02.ply"
filename = "G-PCD/stimuli/D01/cube.ply"
#点群をメッシュにする
mesh = o3d.io.read_triangle_mesh(filename)
#pcd, mesh = ply_to_mesh(filename)

#回転
R = o3d.geometry.get_rotation_matrix_from_yxz([np.pi/3, 0, 0])
print("R:", np.round(R, 7))
R = o3d.geometry.get_rotation_matrix_from_axis_angle([0, np.pi/3, 0])
print("R:", np.round(R, 7))
R = o3d.geometry.get_rotation_matrix_from_quaternion([np.cos(np.pi/6), 0, np.sin(np.pi/6), 0])
print("R:", np.round(R, 7))
mesh_r = copy.deepcopy(mesh)
mesh_r.rotate(R, center=[0, 0, 0])

#並進
t = [0.5, 0.7, 1]
mesh_t = copy.deepcopy(mesh_r).translate(t)
print("Type q to continue.")
o3d.visualization.draw_geometries([mesh, mesh_t])

#回転と並進
T = np.eye(4)
T[:3, :3] = R
T[:3, 3] = t
mesh_t = copy.deepcopy(mesh).transform(T)
print("Type q to continue.")
o3d.visualization.draw_geometries([mesh, mesh_t])

#スケール
mesh_s = copy.deepcopy(mesh_t)
mesh_s.scale(0.5, center=mesh_s.get_center())
print("Type q to continue.")
o3d.visualization.draw_geometries([mesh, mesh_s])

R: [[ 0.5        0.         0.8660254]
 [ 0.         1.         0.       ]
 [-0.8660254  0.         0.5      ]]
R: [[ 0.5        0.         0.8660254]
 [ 0.         1.         0.       ]
 [-0.8660254  0.         0.5      ]]
R: [[ 0.5        0.         0.8660254]
 [ 0.         1.         0.       ]
 [-0.8660254  0.         0.5      ]]
Type q to continue.
Type q to continue.
Type q to continue.


In [15]:
#軸のメッシュを作成
filename = "G-PCD/stimuli/D01/cube.ply"
pcd = o3d.io.read_point_cloud(filename)
print(f"初めの座標 : {np.asarray(pcd.points)}")
print("-"*100)
#回転
R = o3d.geometry.get_rotation_matrix_from_yxz([np.pi/3, 0, 0])
print("R:", np.round(R, 7))
R = o3d.geometry.get_rotation_matrix_from_axis_angle([0, np.pi/3, 0])
print("R:", np.round(R, 7))
R = o3d.geometry.get_rotation_matrix_from_quaternion([np.cos(np.pi/6), 0, np.sin(np.pi/6), 0])
print("R:", np.round(R, 7))
print("-"*100)

#回転
pcd_r = copy.deepcopy(pcd).rotate(R, center=[0, 0, 0])
o3d.visualization.draw_geometries([pcd, pcd_r])
print(f"回転後の座標 : {np.asarray(pcd_r.points)}")
print("-"*100)

#並進
t = [0.5, 0.7, 1]
pcd_t = copy.deepcopy(pcd).translate(t)
print("Type q to continue.")
o3d.visualization.draw_geometries([pcd, pcd_t])
print(f"並進後の座標 : {np.asarray(pcd_t.points)}")
print("-"*100)

#回転と並進
T = np.eye(4)
T[:3, :3] = R
T[:3, 3] = t
pcd_t = copy.deepcopy(pcd_t).transform(T)
print("Type q to continue.")
o3d.visualization.draw_geometries([pcd, pcd_t])
print(f"回転・並進後の座標 : {np.asarray(pcd_t.points)}")
print("-"*100)

#スケール
pcd_s = copy.deepcopy(pcd)
pcd_s.scale(0.5, center=pcd_s.get_center())
print("Type q to continue.")
o3d.visualization.draw_geometries([pcd, pcd_s])
print(f"スケール調整後の座標 : {np.asarray(pcd_s.points)}")
print("-"*100)

初めの座標 : [[0.       0.       0.      ]
 [0.       0.014142 0.      ]
 [0.       0.028284 0.      ]
 ...
 [1.       0.989949 0.961665]
 [1.       0.989949 0.975807]
 [1.       0.989949 0.989949]]
----------------------------------------------------------------------------------------------------
R: [[ 0.5        0.         0.8660254]
 [ 0.         1.         0.       ]
 [-0.8660254  0.         0.5      ]]
R: [[ 0.5        0.         0.8660254]
 [ 0.         1.         0.       ]
 [-0.8660254  0.         0.5      ]]
R: [[ 0.5        0.         0.8660254]
 [ 0.         1.         0.       ]
 [-0.8660254  0.         0.5      ]]
----------------------------------------------------------------------------------------------------
回転後の座標 : [[ 0.          0.          0.        ]
 [ 0.          0.014142    0.        ]
 [ 0.          0.028284    0.        ]
 ...
 [ 1.33282632  0.989949   -0.3851929 ]
 [ 1.34507365  0.989949   -0.3781219 ]
 [ 1.35732098  0.989949   -0.3710509 ]]
-------------------

## 2.4 サンプリング

二次元画像  
- ピクセルが格子状に並んでいる
    - 全ての点が輝度やRGBカラー値といった値を持っている
- ピクセルの数は画像の大きさと直接関係する  
  
三次元点群データ
- 点群データは空間的に不均一に存在している
    - 計測器に近い物体は点と点の距離が小さく、遠い物体の表面上の点と点の距離は大きくなる
    - 見えないところの点は計測できない
    - 点の数が多ければ、解像度が高いとも限らない
  
ボクセル化  
- 二次元画像のピクセルを三次元に拡張したもの
- xyz方向に点が整列している
    - ただ計測点の存在しない箇所は空であるため、疎なデータになっている
        - だから二次元画像ほどデータの解析が容易ではない

#### ボクセル化の処理  
- 点群データ内の全ての点xyz座標が最大最小値を取得する
- これらの値全ての組み合わせからなる8点を頂点とする直方体を作る→2^3=8だから8点
    - これをバウンディボックスという
- これらを一片の長さsで分割する（今回だとs=0.03）
- 次に各点の座標を参照して、ボクセルに割り当てられるインデックスを計算して、ボクセルへと点を追加する
- 最後に、各ボクセル内の点群の座標の平均値を求め、ボクセルごとに0又は１個の新しい点を作成して、点群データを出力する
    - 単純にボクセル化だとこの動作をしない場合もある
    
注意  
単純に等間隔サンプリングだとk=3の時に全体の3分の1の個数の点群データを出力するが、点群データの点の並びは一般的に規則性がない  
即ち、入力点群データの点が空間的に等間隔に並んでない限りは、出力点群データも空間的に等間隔にはならない

In [5]:
#等間隔サンプリング
import sys
import open3d as o3d

#filename = sys.argv[1]
#filename = "../3rdparty/Open3D/examples/test_data/fragment.ply"
filename = "/Users/sotaaraki/Downloads/20221109_work_cube20.ply"
#s = float(sys.argv[2])
s = 3
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

#点群の描画
o3d.visualization.draw_geometries([pcd], zoom=0.3412, front=[0.4257, -0.2125, -0.8975], lookat=[2.6172, 2.0475, 1.532], up=[-0.0694, -0.9768, 0.2024])
downpcd = pcd.voxel_down_sample(voxel_size=s) #パラメータsで指定した長さをボクセルの一辺とし、各ボクセルの中に1点が存在するようにダウンサンプリングをしている
print(downpcd)

#点群の描画
o3d.visualization.draw_geometries([downpcd], zoom=0.3412, front=[0.4257, -0.2125, -0.8975], lookat=[2.6172, 2.0475, 1.532], up=[-0.0694, -0.9768, 0.2024])

Loading a point cloud from /Users/sotaaraki/Downloads/20221109_work_cube20.ply
PointCloud with 1687887 points.
PointCloud with 3531 points.


#### FPS(Farthest Point Sampling)
各部分領域の中心点を求めるために、点群データ全体からなるべく等間隔に離れたk個の点をサンプリングするために使う手法。また距離の定義を任意に設定できるということも特徴。  
また計算コストが高いということが難点である。そのため、学習時の前処理などのオフライン処理に適している。

- https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=623193

原理  
- 初めに1点をランダムに選択します
- この点と他の全ての距離を計算して、最も距離の大きい点を選択する
- この選択された点と他の全ての点との距離を計算する
- 各点について、上記の二つの距離のうち、最小の距離に注目する
- そして、その距離が最大となる点を第三の点として選択する
- 上記の動作を繰り返して、k個の点を選択する

In [6]:
#Farthest Point Sampling(FPS)
import sys
import numpy as np
import open3d as o3d

def l2_norm(a, b):
    return ((a-b)**2).sum(axis=1)

def farthest_point_sampling(pcd, k, metrics=l2_norm):
    indices = np.zeros(k, dtype=np.int32)
    points = np.asarray(pcd.points) #np.asarrayは元の配列も変化する。ちなみにnp.arrayは元の配列は変化しない
    distances = np.zeros((k, points.shape[0]), dtype=np.float32)
    indices[0] = np.random.randint(len(points))
    farthest_point = points[indices[0]]
    min_distances = metrics(farthest_point, points)
    distances[0, :] = min_distances
    for i in range(1, k):
        indices[i] = np.argmax(min_distances)
        farthest_point = points[indices[i]]
        distances[i, :] = metrics(farthest_point, points)
        min_distances = np.minimum(min_distances, distances[i, :])
    pcd = pcd.select_by_index(indices)
    return pcd

#main部
#filename = sys.argv[1]
#filename = "../3rdparty/Open3D/examples/test_data/fragment.ply"
filename = "/Users/sotaaraki/Downloads/20221109_work_cube20.ply"
#k = int(sys.argv[2])
k = 1000
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

o3d.visualization.draw_geometries([pcd], zoom=0.3412, front=[0.4257, -0.2125, -0.8975], lookat=[2.6172, 2.0475, 1.532], up=[-0.0694, -0.9768, 0.2024])
dowpcd = farthest_point_sampling(pcd, k)
print(downpcd)

o3d.visualization.draw_geometries([downpcd], zoom=0.3412, front=[0.4257, -0.2125, -0.8975], lookat=[2.6172, 2.0475, 1.532], up=[-0.0694, -0.9768, 0.2024])

Loading a point cloud from /Users/sotaaraki/Downloads/20221109_work_cube20.ply
PointCloud with 1687887 points.
PointCloud with 3531 points.


#### Poisson Disk Sampling(PDS)
FPSが高速処理ができないことがデメリットであったため、それをできるように別の方法をとったものがこの方法。  ランダムサンプリングに近いが、サンプリングされた2点間距離の最小値をある指定した値以下にならないように制御できる。これによって、出力点群の全ての点が互いにある一定以上の距離で離れており、実用的には、空間的均一性の高いサンプリングを行うことができる  

原理
- 初めに一点をランダムに選択する
- その次の点もランダムに選択する
- 選択される点を中心に半径rの円を考え、円の中にすでにサンプリングされた点があれば。選択された点を削除する
- 上記の操作をk回繰り返す

In [7]:
#Possion Disk Sampling(PDS)
import sys
import open3d as o3d

#filename = sys.argv[1]
#filename = "../3rdparty/Open3D/examples/test_data/knot.ply"
filename = "/Users/sotaaraki/Downloads/20221109_work_cube20.ply"
#k = int(sys.argv[2])
k = 500
print("Loading a triangle mesh from", filename)
mesh = o3d.io.read_triangle_mesh(filename)
print(mesh)

o3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True)

downpcd = mesh.sample_points_poisson_disk(number_of_points=k)
print(downpcd)

o3d.visualization.draw_geometries([downpcd])

Loading a triangle mesh from /Users/sotaaraki/Downloads/20221109_work_cube20.ply
TriangleMesh with 1687887 points and 3375766 triangles.
PointCloud with 500 points.


### 最新の研究事例で用いられるサブサンプリング
- FPS
    - 概形を損なわずにサブサンプリングすることが可能
    - ただO(N^2)の計算量が必要なため、タスクや入力点群によっては利用しにくい場合がある
最新の手法
- ShellNet
    - 点群畳み込みを提案した研究事例　
    - 提案した畳み込み手法と組み合わせた場合には、ランダムに点群をサブサンプリングしたとしても性能が悪化しないことが報告されている　
    - https://arxiv.org/pdf/1908.06295.pdf
- Flex-convolution    
    - 点群畳み込みを提案した研究事例　
    - IDISS(Inversity Density Importance Sub-Sampling)
    - O(N)の計算量になるため、FPSより高速
- USIP
    - 三次元位置合わせのための特徴点検出にサブサンプリングを利用した手法
    - 学習して使う
    - https://openaccess.thecvf.com/content_ICCV_2019/papers/Li_USIP_Unsupervised_Stable_Interest_Point_Detection_From_3D_Point_Clouds_ICCV_2019_paper.pdf
- Modeling Subset Sampling
    - Gamblin-Maxをを用いた確率的なサブサンプリング手法の提案
    - https://arxiv.org/pdf/1904.03375.pdf

#### 外れ値除去
統計的外れ値除去  
- 各点とその近傍点との距離の平均値を求める
- この平均値に基づいてある閾値を定め、近傍点との距離が閾値以上になる値の点を外れ値として除去する
    - nb_neighborsは考慮する近傍点の個数
    - std_ratioは距離の閾値を決定するパラメータ。この値が小さいほど、多くの値を外れ値として除去する
  
半径外れ値除去  
- 各点を中心とするある半径の球内の点を近傍点とみなしたとき、近傍点の個数が閾値未満となる点を外れ値として除去する
    - nb_pointsは近傍点の個数の閾値
    - radiusは球の半径

In [15]:
import sys
import open3d as o3d

def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)
    
    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud], zoom=0.3412, front=[0.4257, -0.2125, -0.8975], lookat=[2.6172, 2.0475, 1.532], up=[-0.0694, -0.9768, 0.2024])
    
#filename = sys.argv[1]
filename = "../3rdparty/Open3D/examples/test_data/fragment.ply"
print("Loading a point cloud from", filename)
pcd = o3d.io.read_point_cloud(filename)
print(pcd)

#統計的外れ値除去
print("Statistical oulier removal")
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
display_inlier_outlier(pcd, ind)

#半径外れ値除去
print("Radius oulier removal")
cl, ind = pcd.remove_radius_outlier(nb_points=16, radius=0.02)
display_inlier_outlier(pcd, ind)

Loading a point cloud from ../3rdparty/Open3D/examples/test_data/fragment.ply
PointCloud with 196133 points.
Statistical oulier removal
Showing outliers (red) and inliers (gray): 
Radius oulier removal
Showing outliers (red) and inliers (gray): 


## 2.5 法線推定
点群が何らかの物体表面という曲面上に分布していると仮定する。そして、各点の法線はその点における接平面に直交するベクトルとして定義される
  
#### 点群からの法線推定
- まず各点の近傍点を求める
- 近傍点群の3次元座標に対して主成分分析を行う
    - 主成分分析は近傍点群の分散共分散を求め、その固有値分解を行う手法
    - 今回は法線ベクトルとして、サンプルの分散が小さい軸を選ぶ
    - 最小固有値を持つ、固有ベクトルを選べば、これが求める法線ベクトルに相当する
    
#### メッシュデータからの法線推定
- 面を構成する辺のうち2本の外積を求めれば、それが面の法線ベクトル
    - 点の法線ベクトルはその点が含まれる全ての面の法線ベクトルの平均値として求められる
    - ただノルムが1になるように正規化する必要がある

In [8]:
import sys
import open3d as o3d
import numpy as np

#filename = sys.argv[1]
#filename = "../3rdparty/Open3D/examples/test_data/knot.ply"
filename = "../../Downloads/Housei_20221027_02.ply"
print("Loading a point cloud from", filename)
mesh = o3d.io.read_triangle_mesh(filename)
print(mesh)
#メッシュでデータを表示
o3d.visualization.draw_geometries([mesh])

pcd = o3d.geometry.PointCloud()
pcd.points = mesh.vertices
#法線推定して、法線データを出力する
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=10.0, max_nn=10))

print(np.asarray(pcd.normals))
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

#メッシュデータで法線推定して、法線データを出力する
mesh.compute_vertex_normals()

print(np.asarray(mesh.triangle_normals))
o3d.visualization.draw_geometries([mesh])

Loading a point cloud from ../../Downloads/Housei_20221027_02.ply
TriangleMesh with 1892989 points and 3785990 triangles.
[[ 7.55854828e-01 -6.54739127e-01  3.93562551e-04]
 [ 7.93366075e-01 -6.08743530e-01  1.25931678e-03]
 [ 8.97173582e-01  4.41383087e-01 -1.61410963e-02]
 ...
 [ 8.04054598e-01  5.94136271e-01 -2.23225329e-02]
 [-1.30311612e-02 -9.99915084e-01 -1.15908583e-04]
 [-2.22253969e-03  6.22443896e-03  9.99978158e-01]]
[[-7.60211939e-01  6.49667478e-01  3.15844560e-03]
 [-7.56097649e-01  6.54443965e-01  4.40924985e-03]
 [-7.59202076e-01  6.50854749e-01 -5.51192962e-04]
 ...
 [ 7.64948960e-01 -3.62610357e-01  5.32322100e-01]
 [ 8.01651966e-01 -3.14820535e-01  5.08175320e-01]
 [-3.71816358e-01 -1.76681225e-01  9.11337666e-01]]
